In [8]:
import numpy as np

All calculations are done in SI (Newtons, Meters, Pascals). I'm also assuming 100% efficiency for the ball screw torque to linear force conversion but in reality it might be around ~90% efficiency. So we will need some overhead on the motor torque.

T = torque applied to bearing in Nm

D = diamater of screw



    T = F1*D/2

    => F1 = 2T/D

    F2/F1 = Pi*D/L

    L = lead (distance per revolution)

    => F2 = F1*Pi*D/L
    
    F2 = (2T/D)*Pi*D/L

    F2 = 2T*Pi/L
    


[Source](https://www.kollmorgen.com/en-us/developer-network/calculating-linear-force-and-torque-ballscrew/)

![ballscrew](ballscrew.png)

In [9]:
D = .016 #16mm
L = .005 #5mm
T = .60 #40Ncm .40Nm

F1 = 2*T/D

F2 = 2*T*np.pi/(L)
print("F1: %.3f N" % (F1))
print("F2: %.3f N" % (F2))

F1: 75.000 N
F2: 753.982 N


With F2 we can find the max area of the cylinder while still producing at leas 10 bar

    Force = Pressure/Area

    A = F/P



In [10]:
P = 10 *10**5 #10bar = 1,000,000Pa
A = F2/P
print("A: %.3f cm^2" % (A*10**4)) # cm^2

A: 7.540 cm^2


If we take about 80% of the calculated area is should give more than enough headroom for the motor. The motor also won't be operating at max torque beacus max torque for a steppor motor is calculated with a static load at essentiall 0rpm. If we assume that the shortest we should realisitcally take for an espresso shot is ~20sec then with a Lead of 5mm, an area of 4cm^2, and a max water volume of 40cc it should take 10 revolutions in 20 sec = .5rps = 30rpm. That is quite slow so the motor should be able to apply max torque even at its fastest brew speed.

In [11]:
P = 10 *10**5 #10bar = 1,000,000Pa
A *= .8
#A = 4 *(10**(-4)) #4cm^2 = 4m^2
#r = .0254
#A = np.pi*r**2
F_req = P*A
print("F_req: %.3f N" % (F_req))

F_req: 603.186 N


Since F2 > F_req the the nema 17 should have enough tourque to generate 10 bar pressure 

In [12]:
r = np.sqrt(A/np.pi)
print(r*100*2)
print("Piston A:\n%8.4f m^2\n%8.3f cm^2" % (A,A*10**4))
print("Piston D:\n%8.3f cm\n%8.3f in" % (r*100*2,r*100*2/2.54))

2.7712812921102037
Piston A:
  0.0006 m^2
   6.032 cm^2
Piston D:
   2.771 cm
   1.091 in


In [20]:
50/(A*10**4)

8.289319952702883

Heat energy equation:

    Q = m * c * dt

Q is the energy in joules required to heat the thing (water). m is the mass. c is the specific heat of the thing. dt is the change in temp

Once again we are doing these calculations with 100% efficiency so we will neeed to have quite a bit of overhead to get good results.


In [35]:
#water
m = 50 #mass g
c = 4.186 #specific heat water J/g C
dt = 73 #chamge in temp 20C to 93 C
Q1 = m*c*dt
print("Water: %d J" % (Q1))

#316 stainless for cylinder and piston

m = 100 #mass g, I got this by multiplying the volume of steel by its density
c = .5 #specific heat 316 stainless J/g C
dt = 73 #chamge in temp 20C to 93 C
Q2 = m*c*dt
print("Steel: %d J" % (Q2))
totalQ = Q1+Q2
print("Total Q: %d J" % (totalQ))


Water: 15278 J
Steel: 3650 J
Total Q: 18928 J


1J = 1w/s (watt/sec)

Im planning on using 40-50W 3d printer cartrige heaters (the ones used to heat the printing element) because they are mega cheap and widley availabel if one breaks/burns out.

Initial plan: use 6*50W = 300W of heating

Using the heat equation from above I got an amound of joules that are required to heat the water and steel tube. Now if i divide the joules by the watts of the heating elements I get the number of seconds it would take to heat the water at 100% effeciency.

j/(j/s) =  s

In [34]:
heat_w = 300 #heating power wats
heat_time = totalQ/heat_w
print("%.1f sec" % heat_time)

63.1 sec


I'm going to assume that the heaters wont be 100% efficient so lets call that 1 minute about 1.5 min due to heat loss from the cylinder and electronics